In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from Apify_Scrapper_Functions import Get_Phone_Number_From_Facebook, Get_Phone_Number_From_Website


### Import scrapped data

In [2]:
CarMotor_total_uncleaned_df = pd.read_parquet("./Staging/Silver/CarMotor_total_uncleaned.parquet", engine="fastparquet")

# Check if UEN is unique
print(CarMotor_total_uncleaned_df["UEN"].is_unique)
print(CarMotor_total_uncleaned_df.shape)

True
(1050, 16)


### Separate W/O Phone Numbers

In [3]:
# Separate rows with and without phone numbers
df_with_phones = CarMotor_total_uncleaned_df[CarMotor_total_uncleaned_df["Phones"].notna()]
df_without_phones = CarMotor_total_uncleaned_df[CarMotor_total_uncleaned_df["Phones"].isna()]

# Store unique phone numbers inside final_df_1
final_df_1 = df_with_phones[df_with_phones["Phones"].duplicated(keep=False) == False]

# Store duplicate phone numbers inside refilter_df_1
refilter_df_1 = df_with_phones[df_with_phones["Phones"].duplicated(keep=False) == True]

# Print shapes
print("Total having Phone numbers:", df_with_phones.shape)
print("Total without Phone numbers:", df_without_phones.shape)
print("Unique phone rows (final_df_1):", final_df_1.shape)
print("Duplicate phone rows (refilter_df_1):", refilter_df_1.shape)


Total having Phone numbers: (536, 16)
Total without Phone numbers: (514, 16)
Unique phone rows (final_df_1): (482, 16)
Duplicate phone rows (refilter_df_1): (54, 16)


In [4]:
print(final_df_1["Phones"].astype(str).is_unique)


True


### Merge refilter_df_1 with df_without_phones

In [5]:
df_without_phones_2 = (
    pd.concat([refilter_df_1, df_without_phones], ignore_index=True)
      .drop_duplicates(subset=["UEN"])
)

df_without_phones_2.shape

(568, 16)

### Separate W/O Facebook

In [6]:
# DataFrame with Facebook (not null and not empty)
df_with_facebook = df_without_phones_2[
    df_without_phones_2["Facebook"].notna() &
    (df_without_phones_2["Facebook"] != "")
]

# DataFrame without Facebook (null or empty)
df_without_facebook = df_without_phones_2[
    df_without_phones_2["Facebook"].isna() |
    (df_without_phones_2["Facebook"] == "")
]

# Check the shapes
print(f"With Facebook: {df_with_facebook.shape}")
print(f"Without Facebook: {df_without_facebook.shape}")

With Facebook: (82, 16)
Without Facebook: (486, 16)


In [7]:
print(df_with_facebook["UEN"].is_unique)


True


In [8]:
df_with_facebook.shape

(82, 16)

### Extract Phone Number of Facebook Pages

In [9]:

unique_phone_df, no_phone_df = Get_Phone_Number_From_Facebook(df_with_facebook)


Processing 82 Facebook pages...
Batch 1/1...
  Original URLs: 82, Unique URLs: 71


[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> Status: RUNNING, Message: 
[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> 2025-11-24T06:06:56.306Z ACTOR: Pulling container image of build ZbEBvgTRDUl564DQQ from registry.
[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> 2025-11-24T06:06:58.098Z ACTOR: Creating container.
[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> 2025-11-24T06:06:58.164Z ACTOR: Starting container.
[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> 2025-11-24T06:06:59.114Z INFO  System info {"apifyVersion":"3.5.1","apifyClientVersion":"2.19.0","crawleeVersion":"3.15.2","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> 2025-11-24T06:06:59.673Z INFO  Results Limit undefined
[apify.facebook-page-contact-information runId:C9lb53ubL903HKf8u] -> 2025-11-24T06:06:59.678Z INFO  CheerioCrawler: Using the old Reques

  Retrieved 70 results from Apify
  Sample item keys: ['url', 'error', 'errorDescription']
  DEBUG: URL https://www.facebook.com/msgcars/... | raw_phone=None
  DEBUG: URL https://www.facebook.com/Carlourssg... | raw_phone=+65 9772 2737
  DEBUG: URL https://www.facebook.com/toppremiummotor... | raw_phone=+65 8188 8999
  DEBUG: URL https://www.facebook.com/profile.php?id=... | raw_phone=+65 6902 6346
  DEBUG: URL https://www.facebook.com/pg/arsrecoverys... | raw_phone=+65 8618 9000
  DEBUG: URL https://www.facebook.com/JLCnTservice/... | raw_phone=+65 6367 1338
  DEBUG: URL https://www.facebook.com/hkllimteammotor... | raw_phone=+65 9242 3895
  DEBUG: URL https://www.facebook.com/Carlourssg... | raw_phone=+65 9772 2737
  DEBUG: URL https://www.facebook.com/pg/automotivere... | raw_phone=+65 8618 9000
  DEBUG: URL https://www.facebook.com/profile.php?id=... | raw_phone=+65 6902 6346
  DEBUG: URL https://www.facebook.com/p/Gemini-Car-De... | raw_phone=+65 8468 1217
  DEBUG: URL https://www

In [10]:
print(unique_phone_df.shape)
print(no_phone_df.shape)


print(unique_phone_df["Phones"].is_unique)
print(unique_phone_df["UEN"].is_unique)

print(no_phone_df["Phones"].is_unique)
print(no_phone_df["UEN"].is_unique)

(21, 16)
(61, 16)
True
True
False
True


In [11]:
no_phone_df.shape

(61, 16)

In [12]:
final_df_2 = unique_phone_df.copy()
final_df_3 = pd.concat([final_df_1, final_df_2], ignore_index=True)

df_without_phones_3 = (
    pd.concat([df_without_facebook, no_phone_df], ignore_index=True)
      .drop_duplicates(subset=["UEN"])
)


print(final_df_3.shape)
print(df_without_phones_3.shape)


(503, 16)
(547, 16)


In [13]:
print(final_df_3["UEN"].is_unique)
print(df_without_phones_3["UEN"].is_unique)

True
True


### Separate W/O Websites

In [14]:
# DataFrame with Websites (not null and not empty)
df_with_websites = df_without_phones_3[
    df_without_phones_3["Website"].notna() &
    (df_without_phones_3["Website"] != "")
]

# DataFrame without Websites (null or empty)
df_without_websites = df_without_phones_3[
    df_without_phones_3["Website"].isna() |
    (df_without_phones_3["Website"] == "")
]

# Check the shapes
print(f"With Websites: {df_with_websites.shape}")
print(f"Without Websites: {df_without_websites.shape}")
print(f"Total from df_without_phones_3: {df_without_phones_3.shape}")

With Websites: (75, 16)
Without Websites: (472, 16)
Total from df_without_phones_3: (547, 16)


### Extract Phone Numbers from Websites

In [15]:
scrapped_from_websites = Get_Phone_Number_From_Website(df_with_websites)

🌐 WEBSITE PHONE NUMBER SCRAPER - COST-OPTIMIZED
📊 Configuration:
   • Batch size: 100 websites (increased for efficiency)
   • Concurrency: 3 browsers (parallel processing)
   • Page timeout: 15s (reduced for speed)
   • Function timeout: 30s (optimized)
   • Retries: 1 (minimize compute waste)
   • Browser: Chromium (lightweight)
   • Proxy: RESIDENTIAL (maintained)
   • Strategy: Contact page → Homepage fallback

🔑 Validating Apify API token...
✅ API Key valid - User: SALESOPS_EPOS
   • Plan: STARTER
   • Credits remaining: Check your dashboard at https://console.apify.com/billing

📋 Total rows in RecordOwl_Leads: 75
📋 Rows with valid websites: 75

🔍 STEP 1: VERIFYING WEBSITE ACCESSIBILITY
Testing HTTP/HTTPS connectivity for all websites...
  ⏭️  https://www.mycareersfuture.gov.sg/job/accounting/claims-estimator-account-executive-flash-automotive-f397e5152df0fb4bc2863518227020aa - Skipped (contains 'mycareersfuture')
  ✅ http://rainbowmotor.co.id/
  ✅ https://goldexchange.sg/
  ⏭️  h

[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> Status: RUNNING, Message: 
[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> 2025-11-24T06:09:47.456Z ACTOR: Pulling container image of build g6G5r98rF5fM6ecm3 from registry.
[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> 2025-11-24T06:09:47.458Z ACTOR: Creating container.
[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> 2025-11-24T06:09:47.496Z ACTOR: Starting container.
[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> 2025-11-24T06:09:47.701Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c ./start_xvfb_and_run_cmd.sh && npm run start:prod --silent
[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> 2025-11-24T06:09:48.411Z INFO  System info {"apifyVersion":"3.4.4","apifyClientVersion":"2.16.0","crawleeVersion":"3.14.1","osType":"Linux","nodeVersion":"v22.19.0"}
[apify.puppeteer-scraper runId:hl5dAd7SAcQX7HbAb] -> 2025-11-24T06:09:48.574Z INFO  Configuring Puppeteer Scraper

  ⏳ Run ID: hl5dAd7SAcQX7HbAb
  📊 Status: SUCCEEDED
  ✅ Retrieved 60 results
    ⚠️  http://rainbowmotor.co.id/: No phones found on homepage
    ✅ https://goldexchange.sg/: 1 phone(s) from homepage
    ⚠️  https://mvision-project.sg/: No phones found on homepage
    ✅ https://assureinsurance.com.sg/: 1 phone(s) from homepage
    ⚠️  https://loyaltyandco.sg/: No phones found on homepage
    ⚠️  https://www.q-garage.org/: Not found in results
    ✅ https://www.antz.com.sg/: 1 phone(s) from contact page
    ✅ https://www.sgcarservices.com/: 1 phone(s) from homepage
    ⚠️  https://carlosparts.com/: Not found in results
    ⚠️  https://detailtherapyapparel.com/: No phones found on homepage
    ⚠️  https://ash-logistics.com/: No phones found on homepage
    ⚠️  https://rex-auto.com/: No phones found on homepage
    ⚠️  https://atelierdehues.com/: No phones found on homepage
    ⚠️  https://phoenixlidar.com/: Not found in results
    ⚠️  https://jiaxing-aircon.sg/: Not found in results
    ⚠

In [20]:
# Ensure Phones column exists and convert to string safely
phones = scrapped_from_websites["Phones"].fillna("").astype(str)

# Compute counts once (very efficient)
counts = phones.value_counts(dropna=False)

# Boolean masks (fast vectorized operations)
mask_unique = phones.isin(counts[counts == 1].index)
mask_non_unique = phones.isin(counts[counts > 1].index)

# Create output DataFrames
unique_df = scrapped_from_websites[mask_unique].copy()
non_unique_df = scrapped_from_websites[mask_non_unique].copy()

# Reset index for cleaner output
unique_df.reset_index(drop=True, inplace=True)
non_unique_df.reset_index(drop=True, inplace=True)


In [22]:
non_unique_df.shape

(68, 15)

In [21]:
unique_df.shape

(11, 15)

### merge with final df

In [23]:
# Ensure columns are strings
final_df_3["UEN"] = final_df_3["UEN"].astype(str)
final_df_3["Phones"] = final_df_3["Phones"].astype(str)
unique_df["UEN"] = unique_df["UEN"].astype(str)
unique_df["Phones"] = unique_df["Phones"].astype(str)

# 1. Identify duplicates (conflicts)
mask_duplicate = (
    unique_df["UEN"].isin(final_df_3["UEN"]) |
    unique_df["Phones"].isin(final_df_3["Phones"])
)

# Rows that cannot be appended
unique_conflicts_df = unique_df[mask_duplicate].copy()

# Rows safe to append
unique_clean_df = unique_df[~mask_duplicate].copy()

# 2. Create final_df_4 (new DataFrame)
final_df_4 = pd.concat([final_df_3, unique_clean_df], ignore_index=True)

# 3. Append conflict rows into non_unique_df
non_unique_df = pd.concat([non_unique_df, unique_conflicts_df], ignore_index=True)

# Clean index
final_df_4.reset_index(drop=True, inplace=True)
non_unique_df.reset_index(drop=True, inplace=True)


In [25]:
non_unique_df.shape

(68, 15)

In [29]:
df_without_phones_4 = pd.concat(
    [df_without_websites, non_unique_df],
    ignore_index=True
)

# Drop duplicate UEN
df_without_phones_4 = df_without_phones_4.drop_duplicates(
    subset=["UEN"],
    keep="first"
).reset_index(drop=True)

df_without_phones_4.shape


(536, 16)

In [30]:
df_without_phones_4["UEN"].is_unique

True

In [ ]:
final_df_4["UEN"].is_unique

In [26]:
final_df_4["UEN"].is_unique

True

In [27]:
final_df_4["Phones"].is_unique

True

### Saving

In [31]:
final_df_4.to_parquet("./Staging/Gold/carmotor_scrapped_2_data.parquet", index=False, engine="fastparquet")

In [32]:
df_without_phones_4.to_parquet("./Staging/Gold/carmotor_scrapped_2_no_contact_data.parquet", index=False, engine="fastparquet")